In [84]:
# Imports

import cadquery as cq
from IPython.display import clear_output, display

In [85]:
# Glass reference


def make_glass_shelf_reference(w: float = 40.0, h: float = 7.8, d: float = 20.0):
    ref = cq.Workplane("XY").rect(w, d).extrude(h)
    ref = ref.edges("<Y").edges("|X").fillet(1.2)
    return ref


clear_output(wait=True)
glass_ref = make_glass_shelf_reference()
glass_ref.export("glass_ref.stl")
display(glass_ref)

In [86]:
def make_badge_front(r: float = 20.0, h: float = 5.0):
    body = cq.Workplane("XY").cylinder(h, r).edges("<Z").fillet(1.2)
    return body


clear_output(wait=True)
display(make_badge_front())

In [87]:
# Holder


def make_holder(
    shelf_thickness: float = 7.8,
    wall_thickness: float = 2.5,
    holder_width: float = 20.0,
    holder_depth: float = 15,
) -> cq.Workplane:
    # When thinking the dimensions of this thing, imagine the sketch laying "flat" on the XZ plane.
    # Basically XZ plane is the plane of the piece of shelf where the holder will be inserted.
    # so holder width is the effective width of the shelf covered by holder
    # holder depth is the depth of the shelf covered by the holder
    # shelf thickness is the thickness of the shelf that will be covered by the holder
    # wall thickness is the thickness of the walls of the holder

    holder_height = shelf_thickness + 2 * wall_thickness
    holder_axis = cq.Workplane("YZ").cylinder(holder_width, holder_height / 2)

    left = -holder_width / 2
    right = holder_width / 2
    bottom = 0
    flat_top = holder_depth
    top = flat_top * 1.25

    holder_body = (
        cq.Workplane("XZ")
        .moveTo(left, bottom)
        .lineTo(right, bottom)
        .lineTo(right, flat_top)
        .spline([(right, flat_top), (0, top), (left, flat_top)])
        .lineTo(left, bottom)
        .close()
        .extrude(holder_height)
        .translate((0, holder_height / 2, 0))
    )

    holder_gap = (
        cq.Workplane("XZ")
        .rect(holder_width, holder_depth * 2)
        .extrude(shelf_thickness)
        .translate((0, holder_height / 2 - wall_thickness, holder_depth))
    )

    return holder_axis.union(holder_body) - holder_gap


clear_output(wait=True)
display(make_holder())

In [88]:
# Merged holder+badge


def make_holder_with_badge(angle: float = 25) -> cq.Workplane:
    holder_wall_thickness = 2.5
    shelf_thickness = 7.8
    axis_offset = (shelf_thickness + 2 * holder_wall_thickness) * 0.2

    holder = (
        make_holder(holder_depth=10)
        .rotate((-1, axis_offset, axis_offset), (1, axis_offset, axis_offset), angle)
        .translate((0, 4, 5))
    )
    badge = make_badge_front(r=37.1 / 2)
    return badge.union(holder)


clear_output(wait=True)
complete_badge = make_holder_with_badge()
display(complete_badge)
_ = complete_badge.export("badge.stl")
